In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Achraf\\Desktop\\MASTER\\Python Avancé\\Mini-Projet\\mlops-fakenews-text-classification\\research'

In [4]:
os.chdir("../")


In [5]:
%pwd


'c:\\Users\\Achraf\\Desktop\\MASTER\\Python Avancé\\Mini-Projet\\mlops-fakenews-text-classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    base_model_path: Path
    trained_model_path: Path
    processed_x_train: Path
    processed_y_train: Path
    processed_x_test: Path
    processed_y_test: Path
    tfidf_vectoriser: Path

    

In [7]:
from fakeNewsClassifier.constants import *
from fakeNewsClassifier.utils.common import read_yaml, create_directories, load_bin


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Achraf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Achraf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        data_preprocessing = self.config.data_preprocessing
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            base_model_path= Path(prepare_base_model.base_model_path),
            trained_model_path=Path(training.trained_model_path),
            processed_x_train= Path(data_preprocessing.processed_x_train),
            processed_y_train= Path(data_preprocessing.processed_y_train),
            processed_x_test= Path(data_preprocessing.processed_x_test),
            processed_y_test= Path(data_preprocessing.processed_y_test),
            tfidf_vectoriser= Path(data_preprocessing.tfidf_vectoriser)

        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import time
from sklearn.linear_model import LogisticRegression
import joblib

In [12]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = load_bin(
            self.config.base_model_path
        )
    
    def get_train_data(self):
        self.X = load_bin(self.config.processed_x_train)
        self.y = load_bin(self.config.processed_y_train)


    @staticmethod
    def save_model(path: Path, model: LogisticRegression):
        joblib.dump(model, path)
    
    def train(self):

        self.model.fit(
            self.X,
            self.y
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model)

In [13]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.get_train_data()
    training.train()
    
except Exception as e:
    raise e

[2025-05-13 01:21:16,515: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-13 01:21:16,519: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-13 01:21:16,520: INFO: common: created directory at: artifacts]
[2025-05-13 01:21:16,522: INFO: common: created directory at: artifacts\training]
[2025-05-13 01:21:16,531: INFO: common: binary file loaded from: artifacts\prepare_base_model\base_model.pkl]
[2025-05-13 01:21:16,577: INFO: common: binary file loaded from: artifacts\data_preprocessing\X_train_tfidf.pkl]
[2025-05-13 01:21:16,581: INFO: common: binary file loaded from: artifacts\data_preprocessing\y_train.pkl]
